In [1]:
%matplotlib inline  
import matplotlib.pyplot as plt   
import pandas as pd
import random
import numpy as np
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook
from time import sleep
import scipy
import operator
import difflib
import math
from IPython.core.display import display,HTML
try:
    import cPickle as pickle     #it is faster than pickle!
except:
    import pickle
    
import unicodedata
import networkx as nx
import itertools
import seaborn as sns   ### https://seaborn.pydata.org/tutorial/categorical.html
    
display(HTML("<style>.container { width:100% !important; }</style>"))  # to make the notebook use the entire width of the browser



import plotly.plotly as py
from plotly.graph_objs import *

# i only need my credentials if i want to plot online --- and send plots to server (limits per day apply!)
#import plotly.tools as tls
#tls.set_credentials_file(username='juliettapc', api_key='deyNIvtOoDZ5PLmrHlhd')  # my plotly account credentials


import pygraphviz
from networkx.drawing.nx_agraph import graphviz_layout




########## to be able to plot offline (without sending the plots to the plotly server every time)
import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

In [ ]:
import plotly
plotly.__version__

In [2]:
path="/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/Dropbox/"
df = pd.read_excel(open(path+'NICO_0511_anonymized.xlsx','rb'),na_values=["NAN","-1"],names = ['folder_id', 'user_id', 'email_domain', 'num_adds', 'num_edits', 'num_deletes', 'num_folder_members', 'major_content_type' ,'major_content_ext',  'age_folder_days' ])



In [4]:
for item in df.email_domain.unique():
    print item,"   "

*montana.edu subdomain    
*tamu.edu subdomain    
*uchc.edu subdomain    
*ucla.edu subdomain    
*ucsd.edu subdomain    
*ucsf.edu subdomain    
*uio.no subdomain    
*umn.edu subdomain    
*unm.edu subdomain    
*usc.edu subdomain    
*waseda.ac.jp subdomain    
2014.hull.ac.uk    
2017.icam.fr    
aa.ufl.edu    
aacc.edu    
aalto.fi    
aamu.edu    
aaschool.ac.uk    
aau.at    
aauj.edu    
abbey.bac.edu    
abdn.ac.uk    
aberdeen.ac.uk    
abet.org    
ac.upc.edu    
acad.umass.edu    
academicimpressions.com    
academyart.edu    
acadiau.ca    
acc.org    
accountancy.smu.edu.sg    
ace.tamut.edu    
aces.edu    
acg.edu    
acomedu.org    
acsw.ru.nl    
actx.edu    
acu.edu    
acu.edu.au    
ad.nmsu.edu    
ad.ucsd.edu    
ad.ufl.edu    
ad.umanitoba.ca    
ad.unc.edu    
adams.edu    
adelaide.edu.au    
adelphi.edu    
adjuncts.belmont.edu    
adler.edu    
adm.monash.edu.au    
admin.cam.ac.uk    
admin.fsu.edu    
admin.rochester.edu    
admin.umass.edu    
admin.usf.e

In [32]:
df[df.user_id == 8151657]#2174241653] #2335433869]

#table[table.column_name == some_value]

,folder_id,user_id,email_domain,num_adds,num_edits,num_deletes,num_folder_members,major_content_type,major_content_ext,age_folder_days
1,2174241653,8151657,*tamu.edu subdomain,256,272,62,2,office_docs,docx,512
2,2476590757,8151657,*tamu.edu subdomain,240,408,108,4,office_docs,docx,322
4,2748274007,8151657,*tamu.edu subdomain,134,344,20,2,office_docs,docx,138
5,434776253,8151657,*tamu.edu subdomain,3536,14604,2268,4,office_docs,docx,1573
6,455530165,8151657,*tamu.edu subdomain,6076,28802,3798,4,pdf,pdf,1549
8,1493785393,8151657,*tamu.edu subdomain,4508,11432,1602,4,design,eps,910
10,181065769,8151657,*tamu.edu subdomain,4524,156,1092,20,office_docs,doc,1943
11,1992175155,8151657,*tamu.edu subdomain,2238,498,84,3,other_docs,tex,619


In [5]:

#focus_university="harvard.edu"           #### harvard   missouri      northwestern   berkeley  ucla.  cambridge    ox.ac.uk
# list_focus_univ=["missouri.edu","harvard.edu","northwestern.edu", "berkeley","ucla.","cambridge","ox.ac.uk","ucdavis","princeton","uchicago","umass","columbia","unizar.es","duke.edu"]
list_focus_univ=["stanford"]
# list_focus_univ=["berkeley"]


G=nx.Graph()

dict_folder_list_domains={}
dict_folder_list_users={}
dict_user_id_domain={}
dict_user_id_edit={}
dict_user_id_del={}
dict_user_id_add={}
dict_user_id_age={}



print df.shape
cont=0
for row in df.iterrows():
        #print row[1]
    folder=row[1]["folder_id"]
    user_id=row[1]["user_id"]
        #print folder_id, user_id
                        
    email=row[1]["email_domain"]
    
    edits=row[1]["num_edits"]
    adds=row[1]["num_adds"]
    deletes=row[1]["num_deletes"]
   
    age=row[1]["age_folder_days"]
   
        
    flag_useful_line=0
    for focus_university in list_focus_univ:                 
        if focus_university in email:
            flag_useful_line =1
            break
                 
    if flag_useful_line ==1:
        try: 
            dict_folder_list_users[folder].append(user_id)
        except:
            dict_folder_list_users[folder]=[]
            dict_folder_list_users[folder].append(user_id)

        try:    
            dict_folder_list_domains[folder].append(email)
        except:            
            dict_folder_list_domains[folder]=[]
            dict_folder_list_domains[folder].append(email)

            
            
            
            
    dict_user_id_domain[user_id]=email     
    try:
        dict_user_id_edit[user_id] +=edits   # ojo!! the records are by folder and by user, so i need to accumulate!
        dict_user_id_del[user_id] +=deletes
        dict_user_id_add[user_id] +=adds 
        
        dict_user_id_age[user_id].append(age) 
        
    except :
        dict_user_id_edit[user_id] =edits
        dict_user_id_del[user_id] =deletes
        dict_user_id_add[user_id] =adds 
        
        dict_user_id_age[user_id]=[]
        dict_user_id_age[user_id].append(age) 

        
        
    cont +=1
    if cont % 10000 ==0:
        print cont
            
print "done with the dict., size:",len(dict_folder_list_users)





for user in  dict_user_id_age:
    dict_user_id_age[user]=np.median(dict_user_id_age[user])



for folder in dict_folder_list_users:
    dict_folder_list_users[folder]=list(set(dict_folder_list_users[folder]))  # remove possible duplicates

    dict_folder_list_domains[folder]=list(set(dict_folder_list_domains[folder]))
    
    
    lista=dict_folder_list_users[folder]
  
    if len(lista)>1:
        
        lista_pares=itertools.combinations(lista, 2)        
        
        for item in lista_pares:
            e1=item[0]
            e2=item[1]
            G.add_edge(e1,e2)
           # print "added edge:",e1, e2
#     else:   # FOR NOW, I DO NOT ADD THE NODES WITH COLLABORATORS OUTSIDE THE UNIVERSITY DOMAINS
#         user_id=lista[0]
        
#         G.add_node(user_id)




print "  N:", len(G.nodes()),"  L:", len(G.edges())    



     

#### the master dict is larger than G.nodes(), and the set_node_attribute does not work in that case, so i get a selection of the dict only with the common nodes:
#######################
print len(G.nodes()), len(dict_user_id_domain)

new_dict_user_id = {k: dict_user_id_domain[k] for k in set(G.nodes()) & set(dict_user_id_domain.keys())}
print len(G.nodes()), len(new_dict_user_id)
# i add the email as node attributes    
nx.set_node_attributes(G, 'email', new_dict_user_id)
############



new_dict_user_id = {k: dict_user_id_edit[k] for k in set(G.nodes()) & set(dict_user_id_edit.keys())}
# i add the number of edits as node attributes    
nx.set_node_attributes(G, 'edits', new_dict_user_id)
############



new_dict_user_id = {k: dict_user_id_del[k] for k in set(G.nodes()) & set(dict_user_id_del.keys())}
# i add the number of deletes as node attributes    
nx.set_node_attributes(G, 'deletes', new_dict_user_id)
############



new_dict_user_id = {k: dict_user_id_add[k] for k in set(G.nodes()) & set(dict_user_id_add.keys())}
# i add the number of additions as node attributes    
nx.set_node_attributes(G, 'adds', new_dict_user_id)
############
     
    
 
new_dict_user_id = {k: dict_user_id_age[k] for k in set(G.nodes()) & set(dict_user_id_age.keys())}
# i add the median age as node attributes 
nx.set_node_attributes(G, 'median_age', new_dict_user_id)
############
 
    
    
    
    
    
    
    
       

print list_focus_univ, "done.  N:", len(G.nodes()),"  L:", len(G.edges())  
   
    
    
    
    
    
    
    

########### i relabel the nodes so the plotly works later (otherwise, it doesnt work!)
mapping={}
cont=0
for n in G.nodes():
    mapping[n]=cont
    cont +=1
    
G=nx.relabel_nodes(G,mapping, copy=False)
print "nodes relabeled"
##############################





###### I get a different node shape for each domain (and i group all subdomains together)
dict_domains_cont={}
cont=0
for node in G.nodes():    
    
    domain=str(G.node[node]['email'])
    for focus_university in list_focus_univ:   
        if focus_university in domain:
            domain=focus_university
            break
    
    if domain in dict_domains_cont:
        G.node[node]['shape']=dict_domains_cont[domain]
    else:
        cont +=1
        dict_domains_cont[domain]=cont
        G.node[node]['shape']=dict_domains_cont[domain]


        

print "domains:",set(dict_domains_cont.keys()),"   shapes",set(dict_domains_cont.values())

(497553, 10)
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
done with the dict., size: 2508
  N: 2092   L: 5076
selecting GC and some other large connected components for plotting
  N: 79   L: 143
  N: 79   L: 143
  N: 134   L: 358
  N: 176   L: 470
  N: 212   L: 616
  N: 246   L: 686
  N: 279   L: 735
  N: 311   L: 897
  N: 342   L: 975
  N: 371   L: 1153
  N: 395   L: 1429
  N: 419   L: 1662
  N: 443   L: 1938
  N: 466   L: 1971
  N: 488   L: 2100
  N: 510   L: 2152
  N: 532   L: 2310
  N: 551   L: 2481
  N: 570   L: 2521
  N: 588   L: 2645
  N: 605   L: 2730
  N: 621   L: 2850
  N: 637   L: 2894
  N: 653   L: 2936
  N: 668   L: 3018
  N: 683   L: 3046
  N: 698   L: 3151
  N: 713   L: 3176
  N: 727   L

In [16]:
# help(py.image)

In [17]:
with open(path+"first_network_"+focus_university+".pickle",'wb') as f:
    %time    pickle.dump(G, f)
print "written:",path+"first_network_"+focus_university+".pickle\n"




with open(path+"dict_position_first_network_"+focus_university+".pickle",'wb') as f:
    %time    pickle.dump(dict_pos, f)
print "written:",path+"dict_position_first_network_"+focus_university+".pickle"
